In [5]:
from CircularBuffer import CircularBuffer
from scipy.io import wavfile
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import sys

sys.path.append('/Users/kevinbrennan/Desktop/Final Project/Python Tests/Reverb (Python)/Comb Filter Reverberator')

#KYW inst
fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final Project/Music-Technology-Final-Project/Sound Examples/KYW Inst.wav')
#ATF Drums
#fs, data = wavfile.read('/Users/kevinbrennan/Desktop/Final Project/Music-Technology-Final-Project/Sound Examples/Around the fur drums.wav')

# Get the bit depth of the audio data (assumes audio_data is a NumPy array)
bit_depth = data.dtype.itemsize * 8

# Calculate the scaling factor for normalization
scaling_factor = 2 ** (bit_depth - 1)  # For signed audio

# Convert audio data to floating-point values and normalize
data = data.astype(np.float32) / scaling_factor

# Verify that the data is now in the range -1 to 1
print(f"Min Value: {np.min(data)}")
print(f"Max Value: {np.max(data)}")

Min Value: -0.9484677314758301
Max Value: 0.9998999834060669


In [6]:
delay_ms = 15 #Delay time in ms
wet_amt = 0.5 #Wet amount (0 -> 1)

# Delay time in samples for creating buffer
delaySamps = (int)((delay_ms/1000) * fs)

# Creating Left and Right buffers
audio_bufferL = CircularBuffer(delaySamps)
audio_bufferR = CircularBuffer(delaySamps)


In [7]:
# Audio output array
audio_out = np.zeros((len(data) + delaySamps, 2))

# Processing Loop
for i in range(len(audio_out)):
    # Take next wet sample from buffer
    delayedL = audio_bufferL.get_sample_from_buffer()
    delayedR = audio_bufferR.get_sample_from_buffer()
    
    # Push next dry sample to buffer (if there are dry samples left)
    if(i < len(data)):
        audio_bufferL.push_sample_to_buffer(data[i, 0])
        audio_bufferR.push_sample_to_buffer(data[i, 1])

    # Combine with dry signal and put in output array
    audio_out[i, 0] = (wet_amt * delayedL) 
    audio_out[i, 1] = (wet_amt * delayedR) 
    # Include dry if there is dry left
    if(i < len(data)):
        audio_out[i, 0] += ((1 - wet_amt) * data[i, 0])
        audio_out[i, 1] += ((1 - wet_amt) * data[i, 1])



In [8]:
sd.play(audio_out, fs)